In [18]:
# predict

from neurotron import Cluster, Matrix, Token, Record, Monitor, Toy, isa, follow
import neurotron.math as nm


In [25]:
class Cells(Cluster):
    """
    >>> Cells()
    |-|-|-|-|-|-|-|-|-|-|
    >>> Cells('Mary')
    """
    def __init__(self,shape=(2,5,2,3),token=None):
        if isa(shape,str):
            tag = shape      # rename
            toy = Toy(shape)
            shape = toy.shape
            token = toy.token
        nm.seed(1)
        cells = super().__init__(*shape,verbose=1)
        self.token = token
        m,n,d,s = shape
        f = [0,0,0] if token is None else token['.']
        self.y = nm.row(nm.zeros(1,m*n),f)
        self.record = Record(self)

    def process(self,seq):
        m,n,d,s = self.shape
        seq = [seq] if isa(seq,str) else seq
        prediction = [seq[0],'->']
        for word in seq:
            mon = Monitor(m,n);
            self.y = nm.row(nm.zeros(1,m*n),token[word])
            self.y = self.step(mon,self.y,word)
            output,predict = self.decode()
            mon.xlabel((n-1)/2,output+' -> '+predict)
            prediction.append(predict)
        return prediction

    def predictive(self,list):
        for k in list:
            cells.X[k] = 1;
            cells._predict.I[k] = Matrix([[.1,-.1,.1,-.1,.1],[0,0,0,0,0]])

    def map(self):
        self._predict.map()
        
    def __str__(self):
        self.record.clear();  
        self.record(self)
        return self.record.pattern()

    def __repr__(self):
        return self.__str__()



In [26]:
#=========================================================================
# setup network
#=========================================================================

token = Token({'Mary': [1,0,0,0,0,0,0,1,1],
               'John': [0,1,0,0,0,0,0,1,1],
               'Lisa': [1,0,0,0,0,0,1,1,0],
               'Andy': [0,1,0,0,0,0,1,1,0],
               'likes':[0,0,1,0,0,0,0,1,1],
               'to':   [0,0,0,1,0,0,0,1,1],
               'sing': [0,0,0,0,1,0,0,1,1],
               'dance':[0,0,0,0,0,1,0,1,1],
               'hike': [0,0,0,0,1,0,1,1,0],
               'climb':[0,0,0,0,1,0,1,0,1],
               '.':    [0,0,0,0,0,0,1,1,1],
              })
    
def setup(token):
    shape = (2,9,8,3);  
    cells = Cells(shape,token)
    cells._predict.clear()

    try:
        cells.connect([0,14,16],[4,14,16])  # Mary -> likes
        cells.connect([4,14,16],[6,14,16])  # likes -> to
        cells.connect([6,14,16],[8,14,16])  # to -> sing
        cells.connect([8,14,16],[12,14,16]) # sing -> .

        cells.connect([2,14,16],[5,15,17])  # John -> likes  
        cells.connect([5,15,17],[7,15,17])  # likes -> to
        cells.connect([7,15,17],[10,15,17]) # to(7,15,17) -> dance(10,15,17)
        cells.connect([10,15,17],[13,15,17])# dance -> .

        cells.connect([0,12,14],[4,14,17])  # Lisa -> likes
        cells.connect([4,14,17],[6,15,16])  # likes -> to
        cells.connect([6,15,16],[8,12,14])  # to -> hike
        cells.connect([8,12,14],[13,15,17]) # hike -> .

        cells.connect([2,12,14],[5,14,17])  # Andy -> likes
        cells.connect([5,14,17],[7,15,16])  # likes -> to
        cells.connect([7,15,16],[9,12,16])  # to -> climb
        cells.connect([9,12,16],[13,15,17]) # climb -> .
    except Synapse as s:
        print(Ansi.R,'*** warning:',str(s),Ansi.N)
        
    return cells

cells = setup(token)


In [52]:
class Train:
    """
    parameter training
    >>> Train()
    Train(Cells(2,5,2,3))
    """
    def __init__(self,cells=None):
        self.memory = []
        self.cells = Cells() if cells is None else cells

    def pattern(self,list):
        """
        >>> Train().pattern([1,0,1,0])
        '1010'
        """
        str = ''
        for item in list: str += '1' if item else '0'
        return str
    
    def hash(self,M):    # hash of a matrix
        """
        Train().hash(Matrix([[1,1,0],[0,1,1]]))
        """
        list = M.list();  str = ''
        m,n = M.shape; sep = ''
        for i in range(m):
            row = list[i]
            str += sep + self.pattern(row); sep = '|'
        return str

    def next(self,M):
        """
        >>> Train().next(Matrix(2,3))
        [1 1 1; 0 0 0]
        >>> Train().next([[0,0,0],[1,1,1]])
        [0 0 1; 1 1 0]
        >>> Train().next([[0,1,1],[1,0,0]])
        [1 0 1; 0 1 0]
        """
        return follow(M)

    def token(self,word):
        """
        >>> Train(Cells('Mary')).token('likes')
        [0, 0, 1, 0, 0, 0, 0, 1, 1]
        """
        return self.cells.token[word]
        
    def index(self,token):
        """
        >>> train = Train(Cells('Mary'))
        >>> train.index(train.token('likes')
        [2, 7, 8]
        """
        idx = []
        for k in len(token):
            if token[k]: idx.append(k)
        return idx
        
    def __call__(self,context,word):
        """
        >>> train = Train(Cells('Mary'))
        """
        print((context,word))
        context = context + ' ' + word  # new context
        return context

    def __str__(self):
        if self.cells is None: return 'Train()'
        return 'Train(Cells(%g,%g,%g,%g))' % self.cells.shape

    def __repr__(self):
        return self.__str__()
"""
how to represent 'Mary'?
=> from token get the columns (indices)
"""
        

"\nhow to represent 'Mary'?\n=> from token get the columns (indices)\n"

In [53]:
train = Train(Cells('Mary')); print(train)
train.next(Matrix(2,3))

Train(Cells(2,5,4,3))


[1 1 1; 0 0 0]

In [54]:
context = train('Mary','likes'); context

('Mary', 'likes')


'Mary likes'

In [56]:
Train(Cells('Mary')).token('likes')

[0, 0, 1, 0, 0, 0, 0, 1, 1]